In [4]:
from datasets import load_from_disk
from datasets import load_dataset, concatenate_datasets
from tqdm.autonotebook import tqdm
from pprint import pprint
import re
from evalplus.data import get_mbpp_plus

In [8]:
mbpp_plus = get_mbpp_plus()

mbpp = load_dataset("google-research-datasets/mbpp", name="full")

mbpp_out = mbpp.filter(
    lambda x: f"Mbpp/{x['task_id']}" not in mbpp_plus,
)

mbpp_in = mbpp.filter(
    lambda x: f"Mbpp/{x['task_id']}" in mbpp_plus,
)

Filter: 100%|██████████| 10/10 [00:00<00:00, 5356.03 examples/s]


In [17]:
load_from_disk("var/mbpp_trajectified")

DatasetDict({
    train: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list', 'react_prompt', 'traj_keys', 'traj_values'],
        num_rows: 374
    })
    test: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list', 'react_prompt', 'traj_keys', 'traj_values'],
        num_rows: 224
    })
    validation: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list', 'react_prompt', 'traj_keys', 'traj_values'],
        num_rows: 39
    })
    prompt: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list', 'react_prompt', 'traj_keys', 'traj_values'],
        num_rows: 10
    })
})

In [10]:
mbpp_out

DatasetDict({
    train: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 266
    })
    test: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 276
    })
    validation: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 51
    })
    prompt: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 3
    })
})

In [16]:
mbpp_in

DatasetDict({
    train: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 108
    })
    test: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 224
    })
    validation: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 39
    })
    prompt: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 7
    })
})

In [14]:
mbpp_in["train"][0]#["task_id"]

{'task_id': 602,
 'text': 'Write a python function to find the first repeated character in a given string.',
 'code': 'def first_repeated_char(str1):\r\n  for index,c in enumerate(str1):\r\n    if str1[:index+1].count(c) > 1:\r\n      return c \r\n  return "None"',
 'test_list': ['assert first_repeated_char("abcabc") == "a"',
  'assert first_repeated_char("abc") == "None"',
  'assert first_repeated_char("123123") == "1"'],
 'test_setup_code': '',
 'challenge_test_list': []}

In [15]:
mbpp_plus["Mbpp/602"]

{'task_id': 'Mbpp/602',
 'prompt': '"""\nWrite a python function to find the first repeated character in a given string.\nassert first_repeated_char("abcabc") == "a"\n"""\n',
 'entry_point': 'first_repeated_char',
 'canonical_solution': '\ndef first_repeated_char(str1):\n  for index, c in enumerate(str1):\n    if str1[:index + 1].count(c) > 1:\n      return c\n  return None\n',
 'base_input': [['abcabc'], ['abc'], ['123123']],
 'atol': 0,
 'plus_input': [[''],
  ['abcdefghijklmnopqrstuvwxyz'],
  ['abcabcxyz'],
  ['ABCaBC'],
  ['a'],
  ['aaa'],
  ['aaaabcabcxyz'],
  ['aaaaaaabcabcxyzz'],
  ['abcdefghinjklmnopqrstuvwxyz'],
  ['aaaa'],
  ['aaaaaaabcabcxyzzaaaa'],
  ['aaaaa'],
  ['aa'],
  ['aaaaaaaabcabcxyzzaaaa'],
  ['aaaaaaaabcabcxyzzaaaaABCaaBC'],
  ['aaaaaaaabcabaaaaaaabcabcxyzzcxyzzaaaa'],
  ['aaaabcabcabcdefghinjklmnopqrstuvwxyzxyz'],
  ['aaaaaaabcabcxyzzaaaaa'],
  ['aaaabcabcabcdABCaBCeafghinjklmnopqrstuvwxyzxyz'],
  ['abcdefgxhinjklmnopqrstuvwxyz'],
  ['aaaaaaabcabaaacxyzzaaaa'],
 

In [ ]:

# mbpp["train"] = mbpp["train"].filter(
#     lambda x: f"Mbpp/{x['task_id']}" not in mbpp_plus,
# )
# # .rename_columns(
# #             {"code": "canonical_solution"}
# #         )

# mbpp["test"] = mbpp["test"].filter(
#     lambda x: f"Mbpp/{x['task_id']}" in mbpp_plus,
# )

# mbpp["validation"] = mbpp["validation"].filter(
#     lambda x: f"Mbpp/{x['task_id']}" not in mbpp_plus,
# )


def trajectify(row):
    # - action: |-
    #         def similar_elements(test_tup1, test_tup2):
    #           res = tuple(set(test_tup1) & set(test_tup2))
    #           return res
    #         res = similar_elements((3, 4, 5, 6), (5, 7, 4, 10))
    #         assert res == (4, 5), "Expected (4, 5) but got {}".format(res)
    #     - observation: "[Executed Successfully with No Output]"
    #     - thought: There is no more AssertionError. I can now submit the solution.
    #     - solution: |-
    #         def similar_elements(test_tup1, test_tup2):
    #           res = tuple(set(test_tup1) & set(test_tup2))
    #           return res
    # Regex pattern to match the assert statement and capture the function call and expected result
    task_id = f"Mbpp/{row['task_id']}"
    code = row["code"].replace("\r\n", "\n").replace("\r", "\n").strip()
    first_test = row["test_list"][0].strip().lstrip()
    pattern = r"assert\s+(\w+\(.*?\))\s*==\s*(.+)"

    # Replacement format
    replacement = r"res = \1\nassert res == \2, \"Expected \2 but got {}\".format(res)"

    # Perform the substitution
    converted_string = (
        re.sub(pattern, replacement, first_test)
        .replace('\\"Expected ', '"Expected ')
        .replace('{}\\"', '{}"')
    )
    code_w_assert = code + "\n" + converted_string.strip()
    prompt = row["text"].strip() + "\n" + first_test
    # (
    #     mbpp_plus[task_id]["prompt"]
    #     .strip('"""')
    #     .strip()
    #     .strip('"""')
    #     .strip()
    #     .replace("\n\nassert", "\nassert")
    # )  # row["text"].strip() + "\n" + first_test
    # print(code_w_assert)
    # print("-----")
    trajectory = [
        {"task": prompt},
        {
            "thought": "I should run a solution on the test case before proposing a solution."
        },
        {"action": code_w_assert},
        {"observation": "[Executed Successfully with No Output]"},
        {"thought": "There is no AssertionError. I can now submit the solution."},
        {"solution": code},
    ]

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {
        # "prompt": row[].strip('"""').strip().strip('"""').strip()
        "react_prompt": prompt,
        "code": code,
        "traj_keys": traj_keys,
        "traj_values": traj_values,
    }


mbpp_trajectified = mbpp.map(trajectify)
# mbpp_trajectified.save_to_disk("var/mbpp_trajectified")

train_concat = concatenate_datasets(
    mbpp.filter(
        lambda x: f"Mbpp/{x['task_id']}" not in mbpp_plus,
    )
    # .rename_columns({"code": "canonical_solution", "text": "prompt"})
    .values()
)

test_concat = concatenate_datasets(
    mbpp.filter(
        lambda x: f"Mbpp/{x['task_id']}" in mbpp_plus,
    )
    # .rename_columns({"code": "canonical_solution", "text": "prompt"})
    .values()
)

# filt = mbpp.filter(
#     lambda x: f"Mbpp/{x['task_id']}" in mbpp_plus,
# )


In [ ]:
from datasets import load_from_disk
from datasets import load_dataset, concatenate_datasets
from tqdm.autonotebook import tqdm
from pprint import pprint
import re
from evalplus.data import get_mbpp_plus

In [ ]:
mbpp_plus = get_mbpp_plus()
mbpp_plus

In [ ]:
print("\n".join(mbpp_plus.keys()))

In [17]:
print(mbpp_plus["Mbpp/603"].keys())

dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'base_input', 'atol', 'plus_input', 'contract', 'assertion'])


In [10]:
print(mbpp_plus["Mbpp/603"]["prompt"].strip('"""')
        .strip()
        .strip('"""')
        .strip()
        .replace("\n\nassert", "\nassert"))

Write a function to get all lucid numbers smaller than or equal to a given integer.
assert get_ludic(10) == [1, 2, 3, 5, 7]


In [23]:
answer = """<solution>

```
import string

def is_pangram(sentence):
    alphabet = string.ascii_lowercase
    sentence_lower = sentence.lower()
    for char in alphabet:
        if char not in sentence_lower:
            return False
    return True
```"""
pattern = r"```(?:python)?\n(.*?)\n```"
match = re.search(pattern, answer, re.DOTALL)
if match:
    print(match.group(1))

import string

def is_pangram(sentence):
    alphabet = string.ascii_lowercase
    sentence_lower = sentence.lower()
    for char in alphabet:
        if char not in sentence_lower:
            return False
    return True


In [14]:
mbpp["train"][2]

{'task_id': 603,
 'text': 'Write a function to get a lucid number smaller than or equal to n.',
 'code': 'def get_ludic(n):\r\n\tludics = []\r\n\tfor i in range(1, n + 1):\r\n\t\tludics.append(i)\r\n\tindex = 1\r\n\twhile(index != len(ludics)):\r\n\t\tfirst_ludic = ludics[index]\r\n\t\tremove_index = index + first_ludic\r\n\t\twhile(remove_index < len(ludics)):\r\n\t\t\tludics.remove(ludics[remove_index])\r\n\t\t\tremove_index = remove_index + first_ludic - 1\r\n\t\tindex += 1\r\n\treturn ludics',
 'test_list': ['assert get_ludic(10) == [1, 2, 3, 5, 7]',
  'assert get_ludic(25) == [1, 2, 3, 5, 7, 11, 13, 17, 23, 25]',
  'assert get_ludic(45) == [1, 2, 3, 5, 7, 11, 13, 17, 23, 25, 29, 37, 41, 43]'],
 'test_setup_code': '',
 'challenge_test_list': []}

In [12]:
mbpp = load_dataset("google-research-datasets/mbpp", name="full")
# .filter(
#     lambda x: f"Mbpp/{x['task_id']}" in mbpp_plus,
# )


In [ ]:


def trajectify(row):
    # - action: |-
    #         def similar_elements(test_tup1, test_tup2):
    #           res = tuple(set(test_tup1) & set(test_tup2))
    #           return res
    #         res = similar_elements((3, 4, 5, 6), (5, 7, 4, 10))
    #         assert res == (4, 5), "Expected (4, 5) but got {}".format(res)
    #     - observation: "[Executed Successfully with No Output]"
    #     - thought: There is no more AssertionError. I can now submit the solution.
    #     - solution: |-
    #         def similar_elements(test_tup1, test_tup2):
    #           res = tuple(set(test_tup1) & set(test_tup2))
    #           return res
    # Regex pattern to match the assert statement and capture the function call and expected result
    task_id = f"Mbpp/{row['task_id']}"
    code = row["code"].replace("\r\n", "\n").replace("\r", "\n").strip()
    first_test = row["test_list"][0].strip().lstrip()
    pattern = r"assert\s+(\w+\(.*?\))\s*==\s*(.+)"

    # Replacement format
    replacement = r"res = \1\nassert res == \2, \"Expected \2 but got {}\".format(res)"

    # Perform the substitution
    converted_string = (
        re.sub(pattern, replacement, first_test)
        .replace('\\"Expected ', '"Expected ')
        .replace('{}\\"', '{}"')
    )
    code_w_assert = code + "\n" + converted_string.strip()
    prompt = (
        mbpp_plus[task_id]["prompt"].strip('"""').strip().strip('"""').strip().replace("\n\nassert", "\nassert")
    )  # row["text"].strip() + "\n" + first_test
    # print(code_w_assert)
    # print("-----")
    trajectory = [
        {"task": prompt},
        {
            "thought": "I should run a solution on the test case before proposing a solution."
        },
        {"action": code_w_assert},
        {"observation": "[Executed Successfully with No Output]"},
        {"thought": "There is no AssertionError. I can now submit the solution."},
        {"solution": code},
    ]

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {
        # "prompt": row[].strip('"""').strip().strip('"""').strip()
        "react_prompt": prompt,
        "code": code,
        "traj_keys": traj_keys,
        "traj_values": traj_values,
    }


mbpp_trajectified = mbpp.map(trajectify)
mbpp_trajectified.save_to_disk("var/mbpp")

In [ ]:
mbpp_trajectified["validation"]["task_id"]

In [ ]:
len(mbpp["validation"])

In [ ]:
mbpp["validation"][41]

In [ ]:
mbpp["validation"].select([41])[0]

In [ ]:
list(range(0,len(mbpp["validation"])))

In [7]:
mbpp_original = load_dataset("google-research-datasets/mbpp", name="full")

In [ ]:
mbpp_plus[""]

In [8]:
mbpp_plus = get_mbpp_plus()
train = concatenate_datasets(
                    mbpp_original.filter(
                        lambda x: f"Mbpp/{x['task_id']}" not in mbpp_plus,
                    )
                    .rename_columns({"code": "canonical_solution", "text": "prompt"})
                    .values(),
                )

In [ ]:
filt = mbpp_original.filter(
                        lambda x: f"Mbpp/{x['task_id']}" in mbpp_plus,
                    )

In [ ]:
filt

In [ ]:
mbpp_original

In [ ]:
mbpp.filter(lambda x: "similar_elements" in x["code"])["prompt"][0]

In [ ]:
def parse_answers(row):
    question = row["question"].strip().replace("’", "'").replace("  ", " ")
    parts = row["answer"].split("####")
    answer = parse_number(parts[-1])
    reasoning = "####".join(parts[:-1]).strip().replace("’", "'").replace("  ", " ")
    return {
        "question": question,
        "answer": answer,
        "reasoning": reasoning,
        "raw_answer": row["answer"],
        "answer_part": parts[-1],
    }


gsm8k = gsm8k.map(parse_answers)

In [ ]:
def react_trajectory(row):
    question = row["question"]
    answer = row["answer"]
    reasoning = row["reasoning"].splitlines()
    trajectory = [{"question": question.strip()}]
    res = answer

    for line in reasoning:
        pattern = (
            r"(?P<pre>(=(\ )?|equals(\ )?)?(\$)?)<<(?P<exp>.*?)=(?P<res>.*?)>>([^\s]*)"
        )
        expressions = re.search(pattern, line)

        if expressions is None:
            trajectory += [
                {"thought": line.strip().replace("  ", " ")},
            ]
        else:
            thought = re.sub(pattern, "", line)
            thought = thought.rstrip(".").rstrip(",")
            exp = expressions.group("exp").strip()
            res = expressions.group("res").strip()

            trajectory += [
                {
                    "thought": f"{thought.strip().replace('  ', ' ')}. I need to calculate {exp}"
                },
                {"action": f"Calculator[{exp}]"},
                {"observation": res},
            ]
    if next(iter(trajectory[-1].keys())) == "observation":
        trajectory.append({"thought": f"The answer is {answer}"})

    trajectory.append({"action": f"Finish[{answer}]"})

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {
        "traj_keys": traj_keys,
        "traj_values": traj_values,
    }


gsm8k["train"] = gsm8k["train"].map(react_trajectory)

In [ ]:
def rewoo_trajectory(row):
    question = row["question"]
    answer = row["answer"]
    reasoning = row["reasoning"].splitlines()
    trajectory = [{"question": question.strip().replace("  ", " ")}]
    res = answer

    for line in reasoning:
        pattern = (
            r"(?P<pre>(=(\ )?|equals(\ )?)?(\$)?)<<(?P<exp>.*?)=(?P<res>.*?)>>([^\s]*)"
        )
        expressions = re.search(pattern, line)

        if expressions is None:
            trajectory += [
                {"thought": line.strip().replace("  ", " ")},
            ]
        else:
            thought = re.sub(pattern, "", line)
            thought = thought.rstrip(".").rstrip(",")
            exp = expressions.group("exp").strip()
            res = expressions.group("res").strip()

            trajectory += [
                {"thought": f"{thought.strip().replace('  ', ' ')}. Calculate {exp}"},
                {"action": f"Calculator[{exp}]"},
                {"observation": res},
            ]

    evidence_counter = 0
    for i in range(len(trajectory)):
        outer = trajectory[i]
        type_event = next(iter(outer.keys()))
        value = next(iter(outer.values()))

        if type_event == "action":
            evidence_counter += 1
        if type_event == "observation":
            for j in range(i + 1, len(trajectory)):
                inner = trajectory[j]
                inner_type_event = next(iter(inner.keys()))
                if inner_type_event == "action":
                    trajectory[j]["action"] = trajectory[j]["action"].replace(
                        value, f"#E{evidence_counter}"
                    )
                elif inner_type_event == "thought":
                    trajectory[j]["thought"] = trajectory[j]["thought"].replace(
                        value, f"#E{evidence_counter}"
                    )
    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {"rewoo_traj_keys": traj_keys, "rewoo_traj_values": traj_values}


gsm8k["train"] = gsm8k["train"].map(rewoo_trajectory)

In [ ]:
gsm8k.save_to_disk("var/gsm8k_proc")

In [22]:
!rm -rf var/gsm8k_proc

In [ ]:
load_from_disk("var/gsm8k_proc")

Sympy tool

Example:
Let x be the cost of the pencil.
If the pen costs 2 times the cost of the pencil, then it costs 2x.
Adding the cost of the pen and pencil we get 2x + x = 3x
Since the total cost is $6 then 3x = $6 therefore x = $6 / 3 = $2
One pen is equal to 2 * x which is 2 * $2 = $4

Use symbolic calculator?

Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
Tho: To make a robe, you need 2 bolts of blue fiber. I need to write 2
Act: Write[2]
Obs: Invalid action. Valid actions are Calculator[<expression>] and Finish[<answer>].
Tho: You also need half as many bolts of white fiber. I need to calculate 2 / 4
Act: Calculator[2 / 4]
Obs: 0.5
Tho: Thus, you need 0.5 bolts of white fiber. I need to write 0.5
Act: Write[0.5]
Obs: Invalid action. Valid actions are Calculator[<expression>] and Finish[<answer>].
Tho: The answer is 2.5
Act: Finish[2.5]

In [ ]:
for x in gsm8kk["train"]:
    if "The moon" in x["question"]:
        pprint(x)

# trajectory bootstrapping

manual conversion of ~5 examples

Cot question/reasoning/answer
ReAct: question/thoughts/observations/answer

Cot question/reasoning/answer query
get React trajectory, if answer matches groundtruth, use, otherwise resample
- can add more examples to prompt to improve results

# json

In [2]:
gsm8k = load_dataset("openai/gsm8k", "main")

In [3]:
def parse_answers(row):
    question = row["question"].strip().replace("’", "'").replace("  ", " ")
    parts = row["answer"].split("####")
    answer = parse_number(parts[-1])
    reasoning = "####".join(parts[:-1]).strip().replace("’", "'").replace("  ", " ")
    return {
        "question": question,
        "answer": answer,
        "reasoning": reasoning,
        "raw_answer": row["answer"],
        "answer_part": parts[-1],
    }


gsm8k = gsm8k.map(parse_answers)

In [ ]:
def react_trajectory(row):
    question = row["question"]
    answer = row["answer"]
    reasoning = row["reasoning"].splitlines()
    trajectory = [{"question": question.strip()}]
    res = answer

    for line in reasoning:
        pattern = (
            r"(?P<pre>(=(\ )?|equals(\ )?)?(\$)?)<<(?P<exp>.*?)=(?P<res>.*?)>>([^\s]*)"
        )
        expressions = re.search(pattern, line)

        if expressions is None:
            trajectory += [
                {"thought": line.strip().replace("  ", " ")},
            ]
        else:
            thought = re.sub(pattern, "", line)
            thought = thought.rstrip(".").rstrip(",")
            exp = expressions.group("exp").strip()
            res = expressions.group("res").strip()

            trajectory += [
                {
                    "thought": f"{thought.strip().replace('  ', ' ')}. I need to calculate {exp}"
                },
                {"action": '{"name": "Calc", "arguments": {"expr": "' + f"{exp}" +'"}}'}, #Calculator[{exp}]"},
                {"observation": res},
            ]
    if next(iter(trajectory[-1].keys())) == "observation":
        trajectory.append({"thought": f"The answer is {answer}"})

    trajectory.append({"action":
                       '{"name": "Finish", "arguments": {"topic": "' + f"{answer}" + '"}}'
                       })
                       #f"Finish[{answer}]"

    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {
        "traj_keys": traj_keys,
        "traj_values": traj_values,
    }


gsm8k["train"] = gsm8k["train"].map(react_trajectory)

In [ ]:
def rewoo_trajectory(row):
    question = row["question"]
    answer = row["answer"]
    reasoning = row["reasoning"].splitlines()
    trajectory = [{"question": question.strip().replace("  ", " ")}]
    res = answer

    for line in reasoning:
        pattern = (
            r"(?P<pre>(=(\ )?|equals(\ )?)?(\$)?)<<(?P<exp>.*?)=(?P<res>.*?)>>([^\s]*)"
        )
        expressions = re.search(pattern, line)

        if expressions is None:
            trajectory += [
                {"thought": line.strip().replace("  ", " ")},
            ]
        else:
            thought = re.sub(pattern, "", line)
            thought = thought.rstrip(".").rstrip(",")
            exp = expressions.group("exp").strip()
            res = expressions.group("res").strip()

            trajectory += [
                {"thought": f"{thought.strip().replace('  ', ' ')}. Calculate {exp}"},
                {"action": '{"name": "Calc", "arguments": {"expr": "' + f"{exp}" +'"}}'},
                {"observation": res},
            ]

    evidence_counter = 0
    for i in range(len(trajectory)):
        outer = trajectory[i]
        type_event = next(iter(outer.keys()))
        value = next(iter(outer.values()))

        if type_event == "action":
            evidence_counter += 1
        if type_event == "observation":
            for j in range(i + 1, len(trajectory)):
                inner = trajectory[j]
                inner_type_event = next(iter(inner.keys()))
                if inner_type_event == "action":
                    trajectory[j]["action"] = trajectory[j]["action"].replace(
                        value, f"#E{evidence_counter}"
                    )
                elif inner_type_event == "thought":
                    trajectory[j]["thought"] = trajectory[j]["thought"].replace(
                        value, f"#E{evidence_counter}"
                    )
    traj_keys = [next(iter(t.keys())) for t in trajectory]
    traj_values = [next(iter(t.values())) for t in trajectory]

    return {"rewoo_traj_keys": traj_keys, "rewoo_traj_values": traj_values}


gsm8k["train"] = gsm8k["train"].map(rewoo_trajectory)

In [ ]:
gsm8k.save_to_disk("var/gsm8k_proc_json")